In [ ]:
import os
from containerStacking import StackingProblem
import time
import roar_net_api.algorithms as alg

In [ ]:
data_dir = "data"
instance_files = [f for f in os.listdir(data_dir) if f.endswith("instance.json")]

for filename in instance_files:
    filepath = os.path.join(data_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data_str = f.read()
    problem = StackingProblem.from_json(data_str, filename)

    print(f'Testing instance {filename}')
    
    #Test random solution
    start_time = time.time()
    solution_random = problem.random_solution()
    print({ 
        'name':'random',
        'objective': solution_random.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_random.is_feasible
    })

    start_time = time.time()
    solution_greedy = alg.greedy_construction(problem)
    print({
        'name':'greedy',
        'objective': solution_greedy.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_greedy.is_feasible
    })

    #Test beam search
    start_time = time.time()
    solution_beam = alg.beam_search(problem,bw = 1)
    print({
        'name':'beam',
        'objective': solution_beam.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_beam.is_feasible
    })

    #Test grasp
    start_time = time.time()
    solution_grasp = alg.grasp(problem, 1)
    print({
        'name':'grasp',
        'objective': solution_grasp.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_grasp.is_feasible
    })

    #Test first improvement
    start_time = time.time()
    solution_fi = alg.first_improvement(problem, solution_greedy)
    print({
        'name':'first improvement',
        'objective': solution_fi.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_fi.is_feasible
    })

    #Test best improvement
    start_time = time.time()
    solution_bi = alg.best_improvement(problem, solution_greedy)
    print({
        'name':'best improvement',
        'objective': solution_bi.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_bi.is_feasible
    })

    #Test simulated annealing -- Takes a LONG time
    # start_time = time.time()
    # solution_sa = alg.sa(problem, solution_greedy,3,1.0)
    # print({
    #     'name':'simulated annealing',
    #     'objective': solution_sa.objective_value(),
    #     'time': time.time() - start_time,
    #     'feasible': solution_bi.is_feasible
    # })